# Displaying Molecular Formulas in Tables and Graphs

Here's how to display formatted molecular formulas in tables and graphs.

In the last blog post, we generated molecular formulas from SMILES strings or RDKit molecules. Once we have those molecular formulas, formatted as Markdown or LaTeX, we might want to display them in tables or graphs.

***Use examples that make sense with [CHNS analysis](https://en.wikipedia.org/wiki/Elemental_analysis#CHNX_analysis)***

In [24]:
from IPython.display import display, Markdown

In [25]:
formulas_markdown = ["C<sub>2</sub>H<sub>6</sub><sup>18</sup>O", "C<sup>13</sup>CH<sub>6</sub><sup>18</sup>O", "<sup>13</sup>C<sub>2</sub>H<sub>6</sub><sup>18</sup>O"]
formulas_latex = ["$C_{2}H_{6}{}^{18}O$", "$C^{13}CH_{6}{}^{18}O$", "$^{13}C_{2}H_{6}{}^{18}O$"]

In [26]:
for formula in formulas_markdown:
    display(Markdown(formula))

C<sub>2</sub>H<sub>6</sub><sup>18</sup>O

C<sup>13</sup>CH<sub>6</sub><sup>18</sup>O

<sup>13</sup>C<sub>2</sub>H<sub>6</sub><sup>18</sup>O

In [27]:
for formula in formulas_latex:
    display(Markdown(formula))

$C_{2}H_{6}{}^{18}O$

$C^{13}CH_{6}{}^{18}O$

$^{13}C_{2}H_{6}{}^{18}O$

NameError: name 'Chem' is not defined